# Train VQ-GAN



In [ ]:
!pip install -q matplotlib
!pip install -q datasets
!pip uninstall -q pytorch_lightning -y
!pip install -q pytorch_lightning==1.7.0
!pip uninstall -q torchmetrics -y
!pip install -q torchmetrics==0.7.0
!pip install -q torchinfo
!pip install -q transformers

from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/path/to/root/

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/TUM/Semester3/Similarity_learning_for_sounds/Code


# Load Dataset

In [ ]:
from data_generation.nsynth import get_nsynth_dataloader

BATCH_SIZE = 4
training_dataset_path = f'data/NSynth/nsynth-STFT-train-52.hdf5'
training_dataloader = get_nsynth_dataloader(training_dataset_path, batch_size=BATCH_SIZE, shuffle=True,
                                              get_latent_representation=False, with_meta_data=False, task="STFT")

# Training

In [ ]:
from model.VQGAN import train_VQGAN

model_name = "your_model_name"

modelConfig = {"in_channels": 3, "hidden_channels": [80, 160], "embedding_dim": 4, "out_channels": 3, "block_depth": 2,
               "attn_pos":  [80, 160], "attn_with_skip": True,
            "num_embeddings": 8192, "commitment_cost": 0.25, "decay": 0.99,
            "norm_type": "groupnorm", "act_type": "swish", "num_groups": 16}

trainingConfig = {"threshold": 1e-3, "ssim_weight": 0.0, "vq_weight": 10.0, "BATCH_SIZE": BATCH_SIZE,
            "lr": 1e-4, "max_iter": 10000, "load_pretrain": False, "model_name": model_name, "device": device,
            "d_lr": 1e-4, "adver_weight": 0.1, "save_steps": 500, "w1": 1.0, "w2": 1.0}

VAE = train_VQGAN(modelConfig, trainingConfig, training_dataloader)

## Apply VQ-GAN

Transform spectral representations into latent representations.

In [ ]:
from model.VQGAN_STFT import get_VQGAN_STFT
from data_generation.nsynth import NSynthDataset

BATCH_SIZE = 64

# load VQVAE
model_name = "your_model_name"

modelConfig = {"in_channels": 3, "hidden_channels": [80, 160], "embedding_dim": 4, "out_channels": 3, "block_depth": 2,
               "attn_pos":  [80, 160], "attn_with_skip": True,
            "num_embeddings": 8192, "commitment_cost": 0.25, "decay": 0.99,
            "norm_type": "groupnorm", "act_type": "swish", "num_groups": 16}
VAE = get_VQGAN_STFT(modelConfig, load_pretrain=True, model_name=model_name)

training_dataset_path = f'data/NSynth/nsynth-STFT-train-52.hdf5'
validating_dataset_path = f'data/NSynth/nsynth-STFT-valid-52.hdf5'
dataset = NSynthDataset(training_dataset_path, get_latent_representation=False, task="STFT")
dataset.create_latent_representation(VAE, quantize=False)